# Generate Data With VINE

We generated the data and ICE curves in R and adjusted some functions from the original Repo (https://github.com/MattJBritton/VINE) to make results comparable with REPID

In [ ]:
from __future__ import division

import time
import datetime

from scipy.spatial.distance import cdist
from scipy.interpolate import interp1d
from scipy.stats import mode
import numpy as np
import pandas as pd

#scikit-learn
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn import datasets
#from sklearn.ensemble.partial_dependence import plot_partial_dependence


#matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  



# Helper Methods

In [ ]:
#from original PyCEBox library
#get the x_values for a given granularity of curve
def _get_grid_points(x, num_grid_points):
    if sorted(list(x.unique())) == [0,1]:
        return [0.,1.], "categorical"
    if num_grid_points is None:
        return x.unique(), "numeric"
    else:
        # unique is necessary, because if num_grid_points is too much larger
        # than x.shape[0], there will be duplicate quantiles (even with
        # interpolation)
        return x.quantile(np.linspace(0, 1, num_grid_points)).unique(), "numeric"

In [ ]:
#from original PyCEBox library
#average the PDP lines (naive method seems to work fine)
def _pdp(ice_data):
    return ice_data.mean(axis=0)

In [ ]:
#transform curves before distance measurement
def _differentiate(series):
        
    dS = np.diff(series)
    return dS

In [ ]:
#method for testing random clusters to ensure that algorithm performance is superior
def _test_random_clusters(ice_data, num_clusters=5):
    temp = np.random.uniform(size=num_clusters)
    distribution = temp/temp.sum()
    cluster_labels = np.random.choice(a = range(num_clusters),\
                                      size=ice_data.shape[0],\
                                      replace=True, p=distribution)
    return cluster_labels

In [ ]:
#interpolate lines to num_grid_points when comparing features for feature-space statistics
def _interpolate_line(x, y, length):
    if len(y) == length:
        return y
    else:
        f = interp1d(x,y, kind="cubic")
        return list(f(np.linspace(x[0], x[-1], num=length, endpoint=True)))

# Generate VINE clusters

In [ ]:
def _get_model_split(columns, model):
    split_feature = columns[model.tree_.feature[0]] if model.tree_.value.shape[0] > 1 else 'none'
    split_val = round(model.tree_.threshold[0],2)
    split_direction = "<=" if model.tree_.value.shape[0] == 1\
    or model.classes_[np.argmax(model.tree_.value[1])] == 1 else ">"
    return split_feature, split_val, split_direction

In [ ]:
def export_adj2(ice_data, data, x_s, column_of_interest, ice_name, num_clusters=5, num_grid_points=20,\
           ice_curves_to_export=100, cluster_method="vine"):
    
    ice_data.columns = x_s
    export_dict = {"features":{}, "distributions":{}}
    ice_data = ice_data.sub(ice_data.mean(axis=1), axis='index')
    pdp_data = _pdp(ice_data)
    export_dict["features"][column_of_interest] = {"feature_name": column_of_interest,
                                                       "x_values": list(x_s),
                                                       "pdp_line": list(pdp_data),
                                                       "clusters":[]
                                                      }
        
    export_dict["features"][column_of_interest]["ice_data"] = np.array(ice_data) 

    #perform clustering
    if cluster_method == "vine":
        ice_data["cluster_label"] = AgglomerativeClustering(n_clusters = num_clusters)\
                .fit(_differentiate(ice_data.values)).labels_
    elif cluster_method == "random":
        ice_data["cluster_label"] = _test_random_clusters(ice_data, num_clusters)
            
    #print(ice_data[x_s].values)      
    ice_data["points"] = ice_data[x_s].values.tolist() 

    #generate all the ICE curves per cluster
    all_curves_by_cluster = ice_data.groupby("cluster_label")["points"].apply(lambda x: np.array(x)) 
        
    splits_first_pass = []
    for cluster_num in range(len(all_curves_by_cluster)):                          
        num_curves_in_cluster = len(all_curves_by_cluster[cluster_num])

        #build model to predict cluster membership
        rdwcY = ice_data["cluster_label"].apply(lambda x: 1 if x==cluster_num else 0)
        #1-node decision tree to get best split for each cluster
        model = DecisionTreeClassifier(criterion="entropy", max_depth=1, presort=False,\
                                           class_weight="balanced")
        model.fit(data, rdwcY)
        split_feature, split_val, split_direction = _get_model_split(data.columns, model)
        splits_first_pass.append({"feature":split_feature, "val":split_val,\
                                      "direction": split_direction, "model": model})
       
    #loop through splits to find duplicates
    duplicate_splits = {}
    for i, split_def in enumerate(splits_first_pass[:-1]):
        for j, split_def_2 in enumerate(splits_first_pass):
            if j<=i or i in duplicate_splits or j in duplicate_splits:
                continue
            elif split_def["feature"] == split_def_2["feature"]\
            and split_def["direction"] == split_def_2["direction"]\
            and (split_def["val"] - split_def_2["val"])/(np.ptp(data.loc[:,split_def["feature"]])) <= 0.1:
                duplicate_splits[j] = i

    ice_data = ice_data.replace(to_replace={"cluster_label":duplicate_splits}, value=None)
    # save ice data to visualize in R
    ice_data.to_csv(ice_name)

    #generate all the ICE curves per cluster
    all_curves_by_cluster = ice_data.groupby("cluster_label")["points"].apply(lambda x: np.array(x)) 
    #average the above to get the mean cluster line
    cluster_average_curves = {key:np.mean(np.array(list(value)), axis=0)\
                            for key,value in all_curves_by_cluster.iteritems()}
        
    for cluster_num in all_curves_by_cluster.keys():                          
        num_curves_in_cluster = len(all_curves_by_cluster[cluster_num])

        #build model to predict cluster membership
        rdwcY = ice_data["cluster_label"].apply(lambda x: 1 if x==cluster_num else 0)
        model = splits_first_pass[cluster_num]["model"]
        predY = model.predict(data) 
        split_feature, split_val, split_direction = _get_model_split(data.columns, model)   

        #get random curves if there are more than 100
    
    #no reason to make the visualization display 1000+ ICE curves for this tool
        if num_curves_in_cluster > ice_curves_to_export:
            individual_ice_samples = [list(x) for x in\
                                        list(all_curves_by_cluster[cluster_num]\
                                        [np.random.choice(num_curves_in_cluster,\
                                                size=ice_curves_to_export, replace=False)])
                                        ]
        else:
            individual_ice_samples = [list(x) for x in\
                                        list(all_curves_by_cluster[cluster_num])\
                                        ]
        
        #add cluster-level metrics to the output dict
        export_dict["features"][column_of_interest]["clusters"].append({
                'accuracy': int(round(100.*metrics.accuracy_score(rdwcY, predY))),
                'precision': int(round(100.*metrics.precision_score(rdwcY, predY))),
                'recall': int(round(100.*metrics.recall_score(rdwcY, predY))),
                'split_feature': split_feature,
                'split_val': split_val,
                'split_direction': split_direction,                   
                'predict_function': model.predict, 
                'cluster_size': num_curves_in_cluster,
                'line': list(cluster_average_curves[cluster_num])
        })  
    return(export_dict)        

# Generate data and export for analysis in R

In [ ]:
# Alternative: create ice curves in R (to be more comparable) and export them again with class labels created by vine
X = pd.read_csv("../data/sim_vine_vs_repid/data.csv", sep=",", decimal=".")


# grid data copied
x_s = np.array([-0.99931637, -0.89416256, -0.78900875, -0.68385494, -0.57870113, -0.47354732, -0.36839350, -0.26323969, -0.15808588, -0.05293207,
  0.05222174,  0.15737556,  0.26252937,  0.36768318,  0.47283699,  0.57799080,  0.68314462,  0.78829843,  0.89345224,  0.99860605])

# read in ice data created in R
ice_data = pd.read_csv("../data/sim_vine_vs_repid/ice_data_Sim1.csv", sep=";", decimal=",", index_col=0)

print(X)

In [ ]:
# 5 clusters
VINE_data = export_adj2(ice_data = ice_data, data = X, x_s = x_s, column_of_interest = "x2", ice_name = "../data/sim_vine_vs_repid/test.csv", num_clusters=5, num_grid_points=20,\
           ice_curves_to_export=500, cluster_method="vine")
VINE_data



In [ ]:
# 2 clusters
VINE_data = export_adj2(ice_data = ice_data, data = X, x_s = x_s, column_of_interest = "x2", ice_name = "../data/result2Clusters.csv", num_clusters=2, num_grid_points=20,\
           ice_curves_to_export=500, cluster_method="vine")